# Toronto Neighborhoods clustering

### In this notebook I will be exploring and clustering different neighborhoods in the Toronoto area

#### Part 1: Defining the data
In this section I will be defining the dataframe used in this project. The data will be taken from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import lxml

In [52]:
#scraping the wikipedia page
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto_df = dfs[0] #the first table of the page contains the required data
toronto_df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Cleaning the data:
* Removing all Postcodes that are not assigned
* Group all neighborhoods in the same postcode
* Rename any "not assigned" neighborhoods with Borough name

In [94]:
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df = toronto_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_df.Neighbourhood.replace('Not assigned', df.Borough, inplace=True)
toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
